In [1]:
#imports

%matplotlib widget
%config InlineBackend.figure_format = 'svg'

from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt
import matplotlib.dates as dts
import numpy as np

import ipywidgets as widgets 
from ipywidgets import interact, interact_manual, interactive, fixed, HBox, VBox, Label, Layout

import datetime as dt
from datetime import datetime

import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [2]:
class Graph:
    def __init__(self, fig, status, notes):
        self.fig = fig
        self.fig.subplots_adjust(top=0.85)
        
        self.ax = self.fig.add_subplot(111)
        self.ax.grid()
        
        self.status = status
        self.notes = notes
        
        self.annot = self.ax.annotate("", xy=(0,0), xytext=(-20, 20),textcoords="offset points",
                     bbox=dict(boxstyle="round", fc="w"),
                     arrowprops=dict(arrowstyle="->"),
                     fontfamily = 'Arial', fontsize = 9)
        self.annot.set_visible(False)
        
    def update(self, ind, init_date, dates, data):
        x,y = self.line.get_data()
        x_i = ind["ind"][0]
        self.annot.xy = (x[ind["ind"][0]], y[ind["ind"][0]])
        text = "{}\n{}\n{}\n{}".format('Date: %s' % str(dts.num2date(dates[x_i]))[0:10],
                                       'Value: %s' % str(data[x_i])[0:5],
                                       'Status: %s' %self.status[x_i],
                                       'Notes: %s' %self.notes[x_i])
        self.annot.set_text(text)
        
    def on_hover(self, event, init_date, dates, data):
        visible = self.annot.get_visible()
        if event.inaxes == self.ax:
            within, ind = self.line.contains(event)
            if within:
                self.update(ind, init_date, dates, data)
                self.annot.set_visible(True)
                self.fig.canvas.draw_idle()
            elif visible:
                self.annot.set_visible(False)
                self.fig.canvas.draw_idle()
                
    def draw_graph(self, dates_arr, data_arr):
        self.line, = self.ax.plot_date(dates_arr, data_arr, 'o')
        self.ax.plot_date(dates_arr, data_arr, linestyle='-', color='#0BB3F1')
    
    def connect_comp(self, init_date, dates, data):
        self.fig.canvas.mpl_connect('motion_notify_event', 
                               lambda event: self.on_hover(event, init_date, dates, data))

In [3]:
#reading data

#read-in, n = number of data points
n = input()

#num format of dates
dates = []
#string format of dates
dateNums = []

#data values (y-values)
rms = []
sfnr = []
rdc = []
status = []
notes = []
#matrix = [[0 for x in range(int(n))] for y in range(2)]
#maybe use matrix?

#number of points in file
max_points = 0

#parsing data from file
with open('data_files/birnfmri3tw_new.dat.txt') as dataF:
    next(dataF)
    for line in dataF:
        max_points+=1
        data = line.split(' ')
        #getting string form of date
        date = data[0]
        #reformatting date into readable format
        reformat = date[0:4]+'/'+date[4:6]+'/'+date[6:8]
        dt = datetime.strptime(reformat, '%Y/%m/%d')
        dateNums.append(str(dt))
        
        #important values
        dates.append(dts.date2num(dt))
        rms.append(float(data[6]))
        sfnr.append(float(data[7]))
        rdc.append(float(data[10]))
        status.append(data[16])
        notes.append(data[36][1:len(data[36])-2])

def update_graph(val, init_date, n_points):
    #shutting down previous plot
    plt.close('all')
    #generating new plot/fig
    fig = plt.figure()
    g = Graph(fig, status, notes)
    sliced_dates = dates[init_date:init_date+n_points]
    if val == 0:
        sliced_rms = rms[init_date:init_date+n_points]
        g.fig.suptitle('Graph of RMS')
        g.ax.set_xlabel('Dates')
        g.ax.set_ylabel('RMS')
        g.draw_graph(sliced_dates, sliced_rms)
        g.connect_comp(init_date, sliced_dates, sliced_rms)
    elif val == 1:
        sliced_sfnr = sfnr[init_date:init_date+n_points]
        g.fig.suptitle('Graph of SFNR')
        g.ax.set_xlabel('Dates')
        g.ax.set_ylabel('SFNR')
        g.draw_graph(sliced_dates, sliced_sfnr)
        g.connect_comp(init_date, sliced_dates, sliced_sfnr)
    elif val == 2:
        sliced_rdc = rdc[init_date:init_date+n_points]
        g.fig.suptitle('Graph of RDC')
        g.ax.set_xlabel('Dates')
        g.ax.set_ylabel('RDC')
        g.draw_graph(sliced_dates, sliced_rdc)
        g.connect_comp(init_date, sliced_dates, sliced_rdc)
    fig.autofmt_xdate()
    
#widgets/runner
#e.g. selecting data set, editing start point/num points
def main():
    date_picker = widgets.Dropdown(
        description = 'Pick initial date:',
        disabled = False,
        style = {'description_width': 'initial'},
        options=[(dateNums[i][0:10], i) for i in range(max_points)]
    )
    
    n_slider = widgets.IntSlider(
        description = 'Number of Points:',
        value = n,
        min = 1,
        max = max_points,
        step = 1,
        continuous_update = False,
        style = {'description_width': 'initial'},
        layout = Layout(width = '381px')
    )
    
    data_set = widgets.Dropdown(
        options=[('RMS', 0), ('SFNR', 1), ('RDC', 2)],
        value=0,
        description='Display:'
    )
    
    w = interactive(update_graph, val=data_set, init_date=date_picker, n_points=n_slider)
    display(w)

#runner
if(__name__ == '__main__'):
    main()

10


interactive(children=(Dropdown(description='Display:', options=(('RMS', 0), ('SFNR', 1), ('RDC', 2)), value=0)…